In [259]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns
from pyscf.scf.uhf import det_ovlp

In [260]:
num_det = 2

In [261]:
# Wavefunction generation
from pyscf import lib, gto, scf
from pyscf import gto, mp, mcscf
def He_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="He 0. 0. 0.0;",
        basis=f"ccecpccpvqz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.UHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, num_det,2)
    # mc.fcisolver.nroots = 2
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc

In [262]:
scf_checkfile = 'rohf.chk'
ci_checkfile = 'casci.chk'
for fname in [scf_checkfile, ci_checkfile]:
    if os.path.isfile(fname):
        os.remove(fname)
mol, mf, mcc = He_casci(scf_checkfile, ci_checkfile)


dict_keys(['mol', '_scf', 'verbose', 'stdout', 'max_memory', 'ncas', 'nelecas', '_ncore', 'fcisolver', 'frozen', 'extrasym', 'e_tot', 'e_cas', 'ci', 'mo_coeff', 'mo_energy', 'mo_occ', 'converged'])


In [263]:
mcc.ci

FCIvector([[ 0.99897607,  0.00661145],
           [ 0.00661145, -0.04426495]])

In [264]:
import pyqmc.api as pyq
from wftools import generate_slater

In [265]:
# Boson WF
import bosonwftools, jastrowspin, bosonslater
import pyscftools, mc
wf0, _ = bosonwftools.generate_boson_wf(mol, mf, mc=mcc)
wave_functions = wf0.wf_factors
for wave in wave_functions:
    if isinstance(wave, bosonslater.BosonWF):
        boson_wf = wave
    if isinstance(wave, jastrowspin.JastrowSpin):
        jastrow_wf = wave       
                


In [266]:
from wftools import generate_slater
wfs = []
num_det = mcc.ci.shape[0] * mcc.ci.shape[1]
for i in range(mcc.ci.shape[0]):
    for j in range(mcc.ci.shape[1]):
        mc0 = copy.copy(mcc)
        mc0.ci = mcc.ci * 0
        mc0.ci[i, j] = 1
        wf0, _ = generate_slater(mol, mf, mc=mc0, optimize_determinants=False)
        wfs.append(wf0)

In [267]:
mc0.ci

FCIvector([[0., 0.],
           [0., 1.]])

In [268]:
configs = pyq.initial_guess(mol, 1)
# configs.configs

In [269]:
configs.configs

array([[[-1.23143631, -0.19862703,  1.64616772],
        [ 0.60530748, -2.30397613, -0.4127462 ]]])

In [270]:
# Boson grad
e = 0
boson_wf.recompute(configs)
g, _, _ = boson_wf.gradient_value(e, configs.electron(e))
gradb = mc.limdrift(np.real(g.T))
valueb = boson_wf.value()
gradb

array([[ 0.19461724,  0.03139119, -0.26016175]])

In [271]:
# Fermion grad
e = 0
_ = [wf.recompute(configs) for wf in wfs]
grads = [np.real(wf.gradient(e, configs.electron(e)).T) for wf in wfs]
vals = [np.real(wf.value()) for wf in wfs]
grad = mc.limdrift(np.mean(grads, axis=0))
grad

array([[ 0.48490057,  0.07821303, -0.64820865]])

In [272]:
# For some configurations, grad and gradb are of opposite sign and close in absolute value, try a few times and find such configurations
# 1. Use the configuration coordinates above, and then use fixed initial configs. 

In [273]:
# Fermion grad enhanced
import gpu
s = int(e >= wfs[0]._nelec[0])
epos = configs.electron(e)
aograd = wfs[0].orbitals.aos("GTOval_sph_deriv1", epos)
mograd = wfs[0].orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, wfs[0]._det_occup[s]]
vec = mograd_vals

ratios = gpu.cp.einsum(
    "ei...dj,idj...->ei...d",
    vec,
    wfs[0]._inverse[s][..., e - s * wfs[0]._nelec[0]],
)
ratios

array([[[ 1.        ,  1.        ]],

       [[ 0.89424967,  0.07555148]],

       [[ 0.14423983,  0.01218623]],

       [[-1.19542109, -0.10099621]]])

In [274]:
# Boson grad enhanced
import gpu
s = int(e >= boson_wf._nelec[0])
epos = configs.electron(e)
aograd = boson_wf.orbitals.aos("GTOval_sph_deriv1", epos)
mograd = boson_wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, boson_wf._det_occup[s]]
vec = mograd_vals

ratios = gpu.cp.einsum(
    "ei...dj,idj...->ei...d",
    vec,
    boson_wf._inverse[s][..., e - s * boson_wf._nelec[0]],
)

upref = gpu.cp.amax(boson_wf._dets[0][1]).real
dnref = gpu.cp.amax(boson_wf._dets[1][1]).real

det_array = (
    boson_wf._dets[0][0, :, boson_wf._det_map[0]]
    * boson_wf._dets[1][0, :, boson_wf._det_map[1]]
    * gpu.cp.exp(
        boson_wf._dets[0][1, :, boson_wf._det_map[0]]
        + boson_wf._dets[1][1, :, boson_wf._det_map[1]]
        - upref
        - dnref
    )
)

numer = 1./2 * gpu.cp.einsum(
    "ei...d,di->ei...",
    ratios[..., boson_wf._det_map[s]],
    # boson_wf.parameters["det_coeff"],
    det_array**2,
    # det_array_noref,
)

numera = 1./2 * gpu.cp.einsum(
    "ei...d,di->ei...",
    ratios[..., boson_wf._det_map[s]],
    # boson_wf.parameters["det_coeff"],
    det_array,
    # det_array_noref,
)

denom = gpu.cp.einsum(
    "di->i...",
    # boson_wf.parameters["det_coeff"],
    det_array**2,
)

denoma = gpu.cp.einsum(
    "di->i...",
    # boson_wf.parameters["det_coeff"],
    det_array,
)

res = numer / denom
res[1:]/res[0]


array([[ 0.19461724],
       [ 0.03139119],
       [-0.26016175]])

In [275]:
ratios[..., boson_wf._det_map[s]]

array([[[ 1.        ,  1.        ,  1.        ,  1.        ]],

       [[ 0.89424967,  0.89424967,  0.07555148,  0.07555148]],

       [[ 0.14423983,  0.14423983,  0.01218623,  0.01218623]],

       [[-1.19542109, -1.19542109, -0.10099621, -0.10099621]]])

In [276]:
det_array

array([[0.11647871],
       [0.41253281],
       [0.28235018],
       [1.        ]])

In [277]:
numer

array([[ 0.63173611],
       [ 0.12294674],
       [ 0.01983095],
       [-0.16435357]])

In [278]:
numera

array([[ 0.90568085],
       [ 0.28497591],
       [ 0.04596577],
       [-0.38095202]])

In [279]:
ratios[..., boson_wf._det_map[s]]

array([[[ 1.        ,  1.        ,  1.        ,  1.        ]],

       [[ 0.89424967,  0.89424967,  0.07555148,  0.07555148]],

       [[ 0.14423983,  0.14423983,  0.01218623,  0.01218623]],

       [[-1.19542109, -1.19542109, -0.10099621, -0.10099621]]])

In [280]:
denoma

array([1.8113617])

In [281]:
upref

-2.2685737911496364

In [282]:
dnref

-2.4139247348177166

In [283]:
boson_wf.parameters['det_coeff']

array([0.25, 0.25, 0.25, 0.25])

In [284]:
s = int(e >= boson_wf._nelec[0])
aograd = boson_wf.orbitals.aos("GTOval_sph_deriv1", configs.electron(e))
mograd = boson_wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, boson_wf._det_occup[s]]
ratios = boson_wf._testrowderiv(e, mograd_vals)
ratios

array([[ 0.5       ],
       [ 0.09730862],
       [ 0.01569559],
       [-0.13008088]])

In [285]:
(ratios[1:] / ratios[0]).T

array([[ 0.19461724,  0.03139119, -0.26016175]])

In [286]:
s = int(e >= wf._nelec[0])
aograd = wf.orbitals.aos("GTOval_sph_deriv1", configs.electron(e))
mograd = wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, wf._det_occup[s]]
ratios = wf._testrowderiv(e, mograd_vals)
ratios

AttributeError: 'MultiplyWF' object has no attribute '_nelec'